<a href="https://colab.research.google.com/github/yiruchen1993/nvidia_gtc_dli_rapids_2020/blob/section_notebooks%2Fdata_manipulation/1_05_grid_converter.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 使用 CuPy 轉換網格坐標

我們大多數的資料皆提供緯度與經度坐標，但對於某些涉及距離的機器學習工作，如找出感染者的地理密度叢集、找出距離指定人士的最近醫院或診所等，使用 笛卡兒網格坐標較方便。我們的道路資料也包含這些坐標。使用針對地區的地圖投影 (此處以 [1936 年英國地形測量集](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid) 的資料為例)，可以讓我們有效率且精準地運算出當地距離。

在此學習筆記中，你將透過使用者定義函數執行資料操作，並產生網格坐標值。如此一來，你將能深入瞭解 NumPy 強大的 GPU 加速直接替代函式庫 [CuPy](https://cupy.chainer.org/)。

## 目標

完成此學習筆記後，你將能夠:

- 使用 CuPy 透過使用者定義函數執行 GPU 加速資料轉換

## 匯入

In [ ]:
import cudf

import numpy as np
import cupy as cp

## 讀取資料

在此學習筆記中，我們將再次載入英國人口資料。現在起，從磁碟讀取資料時，我們會提供具名引數 `dtype` 來指定我們希望欄載入的資料類型。

In [ ]:
%time gdf = cudf.read_csv('./data/pop_1-05.csv', dtype=['float32', 'str', 'str', 'float32', 'float32', 'str'])

CPU times: user 4.42 s, sys: 2.1 s, total: 6.52 s
Wall time: 7.94 s


In [ ]:
gdf.dtypes

age       float32
sex        object
county     object
lat       float32
long      float32
name       object
dtype: object

In [ ]:
gdf.shape

(58479894, 6)

## 使用 NumPy 將經緯度轉換成 OSGB 網格坐標

為了執行坐標轉換，我們會建立函數 `latlong2osgbgrid`，此函數會接受經緯度坐標，並轉換成 [OSGB36 坐標](https://en.wikipedia.org/wiki/Ordnance_Survey_National_Grid):「北邊」與「東邊」值，代表某點在笛卡兒坐標中，距離網格最西南端的距離。

正下方為 `latlong2osgbgrid`，此作業十分仰賴 NumPy:

In [ ]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * np.pi/180
        long = long * np.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                # northing of true origin
    E0 = 400000                 # easting of true origin
    F0 = .9996012717            # scale factor on central meridian
    phi0 = 49 * np.pi / 180     # latitude of true origin
    lambda0 = -2 * np.pi / 180  # longitude of true origin and central meridian
    
    sinlat = np.sin(lat)
    coslat = np.cos(lat)
    tanlat = np.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * np.sin(latdiff) * np.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * np.sin(2*(latdiff)) * np.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * np.sin(3*(latdiff)) * np.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - np.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

### 測試 NumPy 轉換工具

為了測試轉換工具並瞭解其效能，此處我們在英國的大約經緯度範圍內，產生了 10,000,000 組正常分布的隨機坐標。

In [ ]:
%%time
coord_lat = np.random.normal(54, 1, 10000000)
coord_long = np.random.normal(-1.5, .25, 10000000)

CPU times: user 697 ms, sys: 16.4 ms, total: 713 ms
Wall time: 712 ms


In [ ]:
coord_lat.shape

(10000000,)

In [ ]:
coord_lat

array([55.15432395, 54.41939128, 54.20007958, ..., 55.5426888 ,
       55.22090507, 54.68356371])

現在我們會將這些經緯度坐標傳遞到轉換工具中，讓其回傳坐標在 OSGB 網格內的北方與東方值。

In [ ]:
%time grid_n, grid_e = latlong2osgbgrid(coord_lat, coord_long)
print(grid_n[:5], grid_e[:5])

CPU times: user 10 s, sys: 915 ms, total: 10.9 s
Wall time: 10.9 s
[584495.65702285 503029.62736525 478410.18362771 643772.08609626
 386456.23119298] [420302.97789844 456784.72965236 435026.31495171 411326.62638379
 429670.15577338]


## 使用 CuPy 將經緯度轉換成 OSGB 網格坐標

In [ ]:
cp

<module 'cupy' from '/opt/conda/envs/rapids/lib/python3.6/site-packages/cupy/__init__.py'>

In [ ]:
np

<module 'numpy' from '/opt/conda/envs/rapids/lib/python3.6/site-packages/numpy/__init__.py'>

[CuPy](https://cupy.chainer.org/) 是一種類似 NumPy 的矩陣函式庫，通常用來直接替代 NumPy。

在下方的 `latlong2osgbgrid_cupy` 中，我們僅將 `np` 轉換成 `cp`。CuPy 支援多種強大的 GPU 加速工作，而這項能夠將 NumPy 呼叫改成 CuPy 的簡單技術，也是一項十分強大的工具。

In [ ]:
# https://www.ordnancesurvey.co.uk/docs/support/guide-coordinate-systems-great-britain.pdf

def latlong2osgbgrid_cupy(lat, long, input_degrees=True):
    '''
    Converts latitude and longitude (ellipsoidal) coordinates into northing and easting (grid) coordinates, using a Transverse Mercator projection.
    
    Inputs:
    lat: latitude coordinate (north)
    long: longitude coordinate (east)
    input_degrees: if True (default), interprets the coordinates as degrees; otherwise, interprets coordinates as radians
    
    Output:
    (northing, easting)
    '''
    
    if input_degrees:
        lat = lat * cp.pi/180
        long = long * cp.pi/180

    a = 6377563.396
    b = 6356256.909
    e2 = (a**2 - b**2) / a**2

    N0 = -100000                 # northing of true origin
    E0 = 400000                  # easting of true origin
    F0 = .9996012717             # scale factor on central meridian
    phi0 = 49 * cp.pi / 180      # latitude of true origin
    lambda0 = -2 * cp.pi / 180   # longitude of true origin and central meridian
    
    sinlat = cp.sin(lat)
    coslat = cp.cos(lat)
    tanlat = cp.tan(lat)
    
    latdiff = lat-phi0
    longdiff = long-lambda0

    n = (a-b) / (a+b)
    nu = a * F0 * (1 - e2 * sinlat ** 2) ** -.5
    rho = a * F0 * (1 - e2) * (1 - e2 * sinlat ** 2) ** -1.5
    eta2 = nu / rho - 1
    M = b * F0 * ((1 + n + 5/4 * (n**2 + n**3)) * latdiff - 
                  (3*(n+n**2) + 21/8 * n**3) * cp.sin(latdiff) * cp.cos(lat+phi0) +
                  15/8 * (n**2 + n**3) * cp.sin(2*(latdiff)) * cp.cos(2*(lat+phi0)) - 
                  35/24 * n**3 * cp.sin(3*(latdiff)) * cp.cos(3*(lat+phi0)))
    I = M + N0
    II = nu/2 * sinlat * coslat
    III = nu/24 * sinlat * coslat ** 3 * (5 - tanlat ** 2 + 9 * eta2)
    IIIA = nu/720 * sinlat * coslat ** 5 * (61-58 * tanlat**2 + tanlat**4)
    IV = nu * coslat
    V = nu / 6 * coslat**3 * (nu/rho - cp.tan(lat)**2)
    VI = nu / 120 * coslat ** 5 * (5 - 18 * tanlat**2 + tanlat**4 + 14 * eta2 - 58 * tanlat**2 * eta2)

    northing = I + II * longdiff**2 + III * longdiff**4 + IIIA * longdiff**6
    easting = E0 + IV * longdiff + V * longdiff**3 + VI * longdiff**5

    return(northing, easting)

### 測試 CuPy 轉換工具

我們將於此處執行與上方 NumPy 相同的作業，但轉換速度會提升許多。執行下方儲存格後，請嘗試重新執行上方的 NumPy 轉換工具 (包含隨機數生成)，再執行 CuPy 轉換工具，你會發現兩者的差異更大。

In [ ]:
%%time
coord_lat = cp.random.normal(54, 1, 10000000)
coord_long = cp.random.normal(-1.5, .25, 10000000)

CPU times: user 586 ms, sys: 36.5 ms, total: 622 ms
Wall time: 778 ms


In [ ]:
%time grid_n, grid_e = latlong2osgbgrid_cupy(coord_lat, coord_long)
print(grid_n[:5], grid_e[:5])

CPU times: user 2.97 s, sys: 20.2 ms, total: 2.99 s
Wall time: 3.22 s
[320336.74382568 424224.53076329 420476.83605342 599951.29837545
 401514.83836115] [454983.32003193 441771.23520964 419429.47963802 433211.68814224
 442999.97096989]


## 新增網格坐標欄至資料架構

現在，我們將運用 `latlong2osgbgrid_cupy` 將 `northing` 與 `easting` 欄新增至 `gdf`。我們首先要使用 `cp.asarray` 方法來將我們需要的兩個欄 (`lat` 與 `long`) 轉換至 CuPy 陣列。cuDF 與 CuPy 直接透過 `__cuda_array_interface__` 相交，因此我們可以在數毫秒內完成轉換。

In [ ]:
%%time
cupy_lat = cp.asarray(gdf['lat'])
cupy_long = cp.asarray(gdf['long'])

CPU times: user 1.02 ms, sys: 245 µs, total: 1.26 ms
Wall time: 1.27 ms


### 練習: 建立網格欄

在本練習中，你現在已有 `lat` 與 `long` 的 GPU 陣列，接著你將在 `gdf` 中建立 `northing` 與 `easting` 欄。若要這麼做:
- 請在 `latlong2osgbgrid_cupy` 中使用剛建立好的 `cupy_lat` 與 `cupy_long`，將 CuPy 陣列轉換成網格坐標
- 從這些坐標 CuPy 陣列中建立 cuDF 序列，再將 dtype 設為 `float32`
- 將這兩個新的序列新增至 `gdf`，將其命名為 `northing` 與 `easting`

In [ ]:
cupy_lat

array([54.533638, 54.426254, 54.5552  , ..., 51.605267, 51.554646,
       51.57879 ], dtype=float32)

In [ ]:
n_cupy_array

array([515491.89545734, 503572.45342953, 517903.65347138, ...,
       189997.50820025, 184440.25400054, 187036.33415369])

#### 解決方案

In [ ]:
# %load solutions/create_grid_columns
n_cupy_array, e_cupy_array = latlong2osgbgrid_cupy(cupy_lat, cupy_long)
gdf['northing'] = cudf.Series(n_cupy_array).astype('float32')
gdf['easting'] = cudf.Series(e_cupy_array).astype('float32')
print(gdf.dtypes)
gdf.head()


age         float32
sex          object
county       object
lat         float32
long        float32
name         object
northing    float32
easting     float32
dtype: object


,age,sex,county,lat,long,name,northing,easting
0,0.0,m,Darlington,54.533638,-1.524400,Francis,515491.90625,430772.15625
1,0.0,m,Darlington,54.426254,-1.465314,Edward,503572.46875,434685.87500
2,0.0,m,Darlington,54.555199,-1.496417,Teddy,517903.65625,432565.53125
3,0.0,m,Darlington,54.547905,-1.572341,Angus,517059.90625,427660.65625
4,0.0,m,Darlington,54.477638,-1.605995,Charlie,509228.68750,425527.78125


## 下一步

在下一份學習筆記中，我們將回到基本的 cuDF 作業，並著重在資料分析的分組與分類上。

<br>
<div align="center"><h2>請重新啟動核心</h2></div>